In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dl-2024/dadosTextosCientificos.tsv


In [2]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


## extrair carateres especiais, espaços duplos, 

In [3]:
import spacy
# Configurar spaCy para usar GPU
spacy.require_gpu()
print("Usando GPU:", spacy.prefer_gpu())

Usando GPU: True


In [4]:
df = pd.read_csv('/kaggle/input/dl-2024/dadosTextosCientificos.tsv',
                   sep='\t',
                   engine='python',
                   encoding='latin-1')
df

,Título_Público,Descricao_pública
0,Açúcar de frutas e aplicação em bolos como sub...,O uso de inovações tecnológicas auxilia as ind...
1,Desenvolver uma linha de farofas com castanhas...,O uso de inovações tecnológicas auxilia as ind...
2,Desenvolvimento de cobertura líquida para sorvete,O Brasil é considerado um país com a maior bio...
3,Desenvolvimento de conserva de tomate cereja,O Brasil é considerado o país com maior biodiv...
4,Desenvolvimento de massa alimentícia enriqueci...,Nos últimos anos a comunidade científica tem d...
...,...,...
2720,Desenvolvimento de ativos biotecnológicos para...,O avanço das tecnologias para estudos genômico...
2721,SIMA MV POWER,Projeto de PD&I para desenvolvimento de um sis...
2722,Talhonameno Ótimo Klabin,"A Klabin S/A é uma empresa brasileira, de cará..."
2723,Tecnologia de Quantificação de Estoque de Carb...,Com a execução do presente projeto espera-se d...


In [5]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
import random

nlp = spacy.load("pt_core_news_lg")

## Captando entidades de acordo com os temas: DUTOS, BIOATIVOS, BIOCOMBUSTÍVEIS e COVID-19

In [6]:
data_train = [
    ("Desenvolvimento de formulação cosmética veiculando sistema nanoestruturado lipídico contendo pré-bioticos e pós-bióticos", {"entities": [(93, 105, "BIOATIVOS"), (108, 120, "BIOATIVOS")]}),
    ("intuito de incrementar nutricionalmente esses alimentos, agregando compostos bioativos e fibras",{"entities": [(77, 86, "BIOATIVOS")]}),
    ("A empresa BIOATIVOSGROUP celebra em parceria com a Unidade Embrapii da UFABC",{"entities": [(10, 24, "BIOATIVOS")]}),
    ("macrofungos associado ao biofertilizante Acrescent® capazes de controlar fungos", {"entities": [(25, 40, "BIOATIVOS")]}),
    ("podem ser recuperados e transformados em coprodutos de valores nutricionais e bioativos", {"entities": [(78, 87, "BIOATIVOS")]}),
    ("farinha de Ora-Pro-Nobis visando conservação de compostos bioativos", {"entities": [(11, 24, "BIOATIVOS")]}),
    ("Extração e caracterização de compostos bioativos", {"entities": [(29, 48, "BIOATIVOS")]}),
    ("O objetivo do projeto é desenvolver Metodologia de Inspeção Baseada em Risco (IBR) para Sistemas de Dutos Flexíveis para a um planejamento do escopo de inspeção", {"entities": [(100, 115, "DUTOS")]}),
    ("Dispositivo Mitigador de Vibração Induzida por Vórtice para Dutos Submarinos", {"entities": [(60, 76, "DUTOS")]}),
    ("Integridade e extensão de vida de dutos flexíveis", {"entities": [(34, 49, "DUTOS")]}),
    ("sensores de ondas guiadas que possa mapear com precisão adequada a corrosão em áreas críticas de dutos em operação", {"entities": [(96, 101, "DUTOS")]}),
    ("Desenvolvimento de Metodologia para avaliação em Fadiga das amraduras de tração em Risers Flexíveis", {"entities": [(83, 99, "DUTOS")]}),
    ("mecanismos de degradação causados pelos fenômenos de fadiga e corrosão fadiga em risers rígidos e flexíveis", {"entities": [(81, 107, "DUTOS")]}),
    ("Desenvolvimento de tubos bimetálicos", {"entities": [(19, 36, "DUTOS")]}),
    ("O objetivo geral do projeto é quantificar o efeito do liner na resistência ao colapso de dutos submarinos mecanicamente cladeados", {"entities": [(89, 105, "DUTOS")]}),
    ("Inspeção de tubulações com reparos em compósito com técnica MWM-Array", {"entities": [(12, 22, "DUTOS")]}),
    ("propriedades mecânicas e integridade estrutural sob hidrogenação em ligas de níquel com altos teores", {"entities": [(69, 84, "DUTOS")]}),
    ("associado ao ambiente corrosivo demanda dutos de elevada espessura", {"entities": [(40, 45, "DUTOS")]}),
    ("vibração induzida por vórtice que será acoplado a dutos submarinos", {"entities": [(50, 66, "DUTOS")]}),
    ("Desenvolver em escala de laboratório (TRL 3) o conceito para formulação de biocombustível naval.", {"entities": [(76, 90, "BIOCOMBUSTÍVEIS")]}),
    ("geração de açúcares avançados e a produção de biocombustíveis", {"entities": [(45, 60, "BIOCOMBUSTÍVEIS")]}),
    ("Estratégias Enzimáticas para Biocombustiveis avançados", {"entities": [(29, 44, "BIOCOMBUSTÍVEIS")]}),
    ("Desenvolvimento do processo de produção de combustíveis sustentáveis", {"entities": [(43, 68, "BIOCOMBUSTÍVEIS")]}),
    ("Desenvolvimento de me?todos para produc?a?o de bioquerosene", {"entities": [(47, 59, "BIOCOMBUSTÍVEIS")]}),
    ("Desenvolvimento de uma rota tecnológica para a produção de bio-óleo pirolenhoso", {"entities": [(59, 67, "BIOCOMBUSTÍVEIS")]}),
    ("Candidatos a fármacos alvo dirigidos para o tratamento de infecções virais por Coronavírus", {"entities": [(78, 89, "COVID-19")]}),
    ("Produção de proteína S de SARS-COV-2 para desenvolvimento de testes de diagnóstico sorológico.", {"entities": [(26, 36, "COVID-19")]}),
    ("Sistema de Triagem inteligente para diagnóstico da Covid 19", {"entities": [(51, 59, "COVID-19")]}),
    ("Avaliar o potencial de difusão da COVID-19 pelo sistema de esgoto em centros urbanos", {"entities": [(34, 42, "COVID-19")]}),
    ("Desenvolver um método padronizado e otimizado para detecção do vírus SARS-CoV2 em amostras de efluentes", {"entities": [(68, 77, "COVID-19")]}),
    ("O aumento da temperatura corporal é um dos indicadores para o diagnóstico de COVID19", {"entities": [(77, 84, "COVID-19")]}),
    ("desenvolvendo um kit de teste quantitativo para monitoramento da COVID", {"entities": [(65, 70, "COVID-19")]}),
    ("triagem e tratamento do covid 19 com uso de inteligência artificial", {"entities": [(24, 32, "COVID-19")]}),
    ("Desenvolvimento de projeto de instrumentação e automação para sistema de higienização e sanitização de gases do Aerador-Filtro Anti-COVID", {"entities": [(112, 137, "COVID-19")]})
]

## Treinamento para reconhecer as entidades

In [13]:
#modelo spaCy vazio para português
nlp = spacy.blank("pt")
ner = nlp.add_pipe("ner")

# rótulos 
for _, annotations in data_train:
    for ent in annotations["entities"]:
        ner.add_label(ent[2])

# Inicializar o otimizador
optimizer = nlp.begin_training()

# Número máximo de iterações
MAX_ITERATIONS = 100
EARLY_STOPPING_THRESHOLD = 0.01  # Valor da loss para interrupção antecipada

# Treinamento
for iteration in range(MAX_ITERATIONS):
    random.shuffle(data_train)  # Embaralhar os dados
    losses = {}

    # Criar batches para treinamento
    batches = minibatch(data_train, size=compounding(4.0, 32.0, 1.001))

    for batch in batches:
        examples = []
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            try:
                example = Example.from_dict(doc, annotations)  # Verificar desalinhamento
                examples.append(example)
            except ValueError as e:
                print(f"Skipping misaligned entities in: '{text}' - {e}")

        # Atualizar o modelo
        nlp.update(examples, sgd=optimizer, drop=0.35, losses=losses)

    print(f"Iteration {iteration} Losses: {losses}")

    # Verificar interrupção antecipada
    if losses.get("ner", 0) < EARLY_STOPPING_THRESHOLD:
        print(f"Early stopping at iteration {iteration}")
        break

[2024-12-10 22:54:58,374] [INFO] Created vocabulary
[2024-12-10 22:54:58,375] [INFO] Finished initializing nlp object


Iteration 0 Losses: {'ner': 340.02318942919374}
Iteration 1 Losses: {'ner': 105.15050974596033}
Iteration 2 Losses: {'ner': 56.061860582915585}
Iteration 3 Losses: {'ner': 50.20962831809413}
Iteration 4 Losses: {'ner': 45.41712229658356}
Iteration 5 Losses: {'ner': 39.233577073310585}
Iteration 6 Losses: {'ner': 39.850412801348895}
Iteration 7 Losses: {'ner': 41.35663118632958}
Iteration 8 Losses: {'ner': 47.70563071272673}
Iteration 9 Losses: {'ner': 34.021011125986355}
Iteration 10 Losses: {'ner': 29.573873683800528}
Iteration 11 Losses: {'ner': 27.138512237150497}
Iteration 12 Losses: {'ner': 23.42751169325361}
Iteration 13 Losses: {'ner': 26.323659704035656}
Iteration 14 Losses: {'ner': 14.816519659021232}
Iteration 15 Losses: {'ner': 19.41776803421229}
Iteration 16 Losses: {'ner': 11.670568890795323}
Iteration 17 Losses: {'ner': 14.008300000391813}
Iteration 18 Losses: {'ner': 12.711783663130415}
Iteration 19 Losses: {'ner': 11.678893031066016}
Iteration 20 Losses: {'ner': 10.5674

In [14]:
# Salve o modelo
nlp.to_disk("costum_ner")
print("Modelo NER salvo.")

Modelo NER salvo.


## Verificando modelo NER personalizado para as entidades criadas

In [22]:
# Carregue o modelo treinado
nlp = spacy.load("costum_ner")

# Exemplo de textos para extração de entidades
texts = [
    "O projeto tem como objetivo otimizar e escalonar a tecnologia de produção da proteína S do coronavírus SARS-COV-2, para que as empresas parceiras desenvolvam testes para diagnóstico sorológico de COVID-19, assim como estabelecer um painel de soros/plasmas positivos e negativos para COVID-19, que permita validar os testes diagnóstico e registrá-los junto à ANVISA",
    "Desenvolvimento de me?todos para produc?a?o de bioquerosene/hidrocarbonetos verdes",
    "Estabelecer uma metodologia consistente para avaliação do fenômeno de fadiga desenvolvido nos arames que constituem as armaduras de tração de dutos flexíveis",
    "Desenvolvimento de formulação cosmética veiculando sistema nanoestruturado lipídico contendo pré e pós-bióticos (vitaminas e ácidos graxos de cadeia curta) a fim de promoverem o equilíbrio da microbiota da pele"
]

# Extração de entidades
entity_data = []
for text in texts:
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    print(f"Texto: {text}\nEntidades: {entities}\n")
    entity_data.append((text, entities))


Texto: O projeto tem como objetivo otimizar e escalonar a tecnologia de produção da proteína S do coronavírus SARS-COV-2, para que as empresas parceiras desenvolvam testes para diagnóstico sorológico de COVID-19, assim como estabelecer um painel de soros/plasmas positivos e negativos para COVID-19, que permita validar os testes diagnóstico e registrá-los junto à ANVISA
Entidades: [('SARS-COV-2', 'COVID-19'), ('COVID-19', 'COVID-19'), ('COVID-19', 'COVID-19'), ('ANVISA', 'BIOATIVOS')]

Texto: Desenvolvimento de me?todos para produc?a?o de bioquerosene/hidrocarbonetos verdes
Entidades: [('bioquerosene', 'BIOCOMBUSTÍVEIS')]

Texto: Estabelecer uma metodologia consistente para avaliação do fenômeno de fadiga desenvolvido nos arames que constituem as armaduras de tração de dutos flexíveis
Entidades: [('dutos flexíveis', 'DUTOS')]

Texto: Desenvolvimento de formulação cosmética veiculando sistema nanoestruturado lipídico contendo pré e pós-bióticos (vitaminas e ácidos graxos de cadeia curta)

In [23]:
doc = nlp("Desenvolvimento de compostos bioativos e dutos submarinos.")
for ent in doc.ents:
    print(f"Entidade: {ent.text}, Rótulo: {ent.label_}")

Entidade: compostos bioativos, Rótulo: BIOATIVOS
Entidade: dutos submarinos, Rótulo: DUTOS


In [25]:
import os
os.environ["WANDB_DISABLED"] = "true"

## Enriquecendo dataset

In [36]:
# Carregar o modelo NER treinado
nlp = spacy.load("costum_ner")

# Função para enriquecer textos com entidades extraídas
def enrich_with_entities(text):
    if pd.isna(text) or not isinstance(text, str):  # Tratar valores nulos e tipos inesperados
        return text
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return text + " " + " ".join(entities)  # Concatenar texto original com entidades

# Aplicar a função para enriquecer os textos
df["enriched_text"] = df["Descricao_pública"].apply(enrich_with_entities)

In [37]:
df

,Título_Público,Descricao_pública,enriched_text
0,Açúcar de frutas e aplicação em bolos como sub...,O uso de inovações tecnológicas auxilia as ind...,O uso de inovações tecnológicas auxilia as ind...
1,Desenvolver uma linha de farofas com castanhas...,O uso de inovações tecnológicas auxilia as ind...,O uso de inovações tecnológicas auxilia as ind...
2,Desenvolvimento de cobertura líquida para sorvete,O Brasil é considerado um país com a maior bio...,O Brasil é considerado um país com a maior bio...
3,Desenvolvimento de conserva de tomate cereja,O Brasil é considerado o país com maior biodiv...,O Brasil é considerado o país com maior biodiv...
4,Desenvolvimento de massa alimentícia enriqueci...,Nos últimos anos a comunidade científica tem d...,Nos últimos anos a comunidade científica tem d...
...,...,...,...
2720,Desenvolvimento de ativos biotecnológicos para...,O avanço das tecnologias para estudos genômico...,O avanço das tecnologias para estudos genômico...
2721,SIMA MV POWER,Projeto de PD&I para desenvolvimento de um sis...,Projeto de PD&I para desenvolvimento de um sis...
2722,Talhonameno Ótimo Klabin,"A Klabin S/A é uma empresa brasileira, de cará...","A Klabin S/A é uma empresa brasileira, de cará..."
2723,Tecnologia de Quantificação de Estoque de Carb...,Com a execução do presente projeto espera-se d...,Com a execução do presente projeto espera-se d...


## Criando labels de acordo com os assuntos escolhidos e treinando classificador

In [39]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import spacy

# Configurações gerais
MODEL_NAME = "neuralmind/bert-base-portuguese-cased"  # Modelo BERTimbau
BATCH_SIZE = 8
NUM_EPOCHS = 3
MAX_LEN = 128

# Verificar se GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Carregar o modelo NER
nlp = spacy.load("costum_ner")

# Função para enriquecer textos com entidades extraídas
def enrich_with_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents]
    return text + " " + " ".join(entities)

# Dados de exemplo (substituir por dados reais)
data = {
    "train": [
        {"text": "Os dutos submarinos precisam de manutenção urgente.", "label": 0},
        {"text": "Biocombustíveis são o futuro da energia sustentável.", "label": 1},
        {"text": "Os bioativos estão sendo usados em novas formulações cosméticas.", "label": 2},
        {"text": "A pandemia de COVID-19 impactou a economia global.", "label": 3},
    ],
    "validation": [
        {"text": "Dutos flexíveis são utilizados em ambientes corrosivos.", "label": 0},
        {"text": "A extração de bioativos ajuda na saúde.", "label": 2},
    ],
}

# Enriquecer os textos com entidades
for split in data:
    for item in data[split]:
        item["text"] = enrich_with_entities(item["text"])

# Convertendo os dados para DatasetDict
train_dataset = Dataset.from_list(data["train"])
validation_dataset = Dataset.from_list(data["validation"])
dataset = DatasetDict({"train": train_dataset, "validation": validation_dataset})

# Carregando o tokenizer do modelo BERTimbau
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Função para tokenizar os textos
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

# Tokenizando os datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Configurar colunas para treinamento
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Carregando o modelo pré-treinado com número de classes definido
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=4)  # 4 classes
model.to(device)  # Move o modelo explicitamente para a GPU

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
)

# Função para calcular métricas de avaliação
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Configurar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Treinar o modelo
trainer.train()

# Avaliação final
metrics = trainer.evaluate()
print("Métricas de avaliação:", metrics)

Usando dispositivo: cuda


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_23/2740910689.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_app

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.302347,0.500000,0.500000,0.500000,0.500000
2,No log,1.279306,0.500000,0.250000,0.500000,0.333333
3,No log,1.277607,0.500000,0.250000,0.500000,0.333333


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarn

Métricas de avaliação: {'eval_loss': 1.277606725692749, 'eval_accuracy': 0.5, 'eval_precision': 0.25, 'eval_recall': 0.5, 'eval_f1': 0.3333333333333333, 'eval_runtime': 0.0704, 'eval_samples_per_second': 28.408, 'eval_steps_per_second': 14.204, 'epoch': 3.0}


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Testando modelo

In [40]:
# Predição em novos textos
new_texts = [
    "Os bioativos estão em alta na indústria farmacêutica.",
    "Dutos rígidos são amplamente utilizados em engenharia offshore.",
    "A pandemia de COVID-19 levou ao desenvolvimento de vacinas.",
    "Biocombustíveis reduzem a emissão de carbono."
]

# Enriquecer os novos textos com entidades
new_texts_enriched = [enrich_with_entities(text) for text in new_texts]

# Tokenizar os textos
new_encodings = tokenizer(new_texts_enriched, padding="max_length", truncation=True, max_length=MAX_LEN, return_tensors="pt")
new_encodings = {key: val.to(device) for key, val in new_encodings.items()}  # Move as entradas para GPU

# Modelo em modo de avaliação
model.eval()
with torch.no_grad():
    outputs = model(**new_encodings)
    predictions = torch.argmax(outputs.logits, dim=1)

print("Textos:", new_texts)
print("Predições:", predictions.cpu().numpy())

Textos: ['Os bioativos estão em alta na indústria farmacêutica.', 'Dutos rígidos são amplamente utilizados em engenharia offshore.', 'A pandemia de COVID-19 levou ao desenvolvimento de vacinas.', 'Biocombustíveis reduzem a emissão de carbono.']
Predições: [2 1 1 1]


### Modelo ficou ok, conseguiu prever a maioria dos clusters de acordo com as entidades nomeadas